In [1]:
# Import Necessary Libraries
import pandas as pd
import numpy as np
from numpy import mean
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

from xgboost import XGBClassifier
# load the dataset
df = pd.read_csv("/content/drive/MyDrive/dataset_full.csv")
df.head(3)

x = df.drop('phishing', axis = 'columns')
y = df["phishing"]

cor_matrix = x.corr().abs()
print(cor_matrix)

upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(bool))
print(upper_tri)

to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.70)]
print(); print(to_drop)

df1 = x.drop(['qty_and_url', 'domain_length', 'qty_dot_directory', 'qty_hyphen_directory', 'qty_slash_directory', 'qty_questionmark_directory', 'qty_equal_directory', 'qty_at_directory', 'qty_and_directory', 'qty_exclamation_directory', 'qty_space_directory', 'qty_tilde_directory', 'qty_comma_directory', 'qty_plus_directory', 'qty_asterisk_directory', 'qty_hashtag_directory', 'qty_dollar_directory', 'qty_percent_directory', 'directory_length', 'qty_dot_file', 'qty_hyphen_file', 'qty_underline_file', 'qty_slash_file', 'qty_questionmark_file', 'qty_equal_file', 'qty_at_file', 'qty_and_file', 'qty_exclamation_file', 'qty_space_file', 'qty_tilde_file', 'qty_comma_file', 'qty_plus_file', 'qty_asterisk_file', 'qty_hashtag_file', 'qty_dollar_file', 'qty_percent_file', 'qty_dot_params', 'qty_underline_params', 'qty_questionmark_params', 'qty_equal_params', 'qty_at_params', 'qty_and_params', 'qty_exclamation_params', 'qty_space_params', 'qty_tilde_params', 'qty_comma_params', 'qty_plus_params', 'qty_asterisk_params', 'qty_hashtag_params', 'qty_dollar_params', 'params_length', 'tld_present_params', 'qty_params', 'email_in_url'], axis=1)
print(); print(df1.head())

# Prepare cross-validation procedure
crossVal = KFold(n_splits = 10, random_state = 1, shuffle = True)

from sklearn.preprocessing import StandardScaler
new_x = StandardScaler().fit_transform(df1)

                      qty_dot_url  ...  url_shortened
qty_dot_url              1.000000  ...       0.059222
qty_hyphen_url           0.109737  ...       0.010346
qty_underline_url        0.127851  ...       0.012158
qty_slash_url            0.246201  ...       0.001125
qty_questionmark_url     0.409549  ...       0.012847
...                           ...  ...            ...
tls_ssl_certificate      0.065105  ...       0.037850
qty_redirects            0.051842  ...       0.049475
url_google_index         0.004035  ...       0.001785
domain_google_index      0.002490  ...       0.000045
url_shortened            0.059222  ...       1.000000

[111 rows x 111 columns]
                      qty_dot_url  ...  url_shortened
qty_dot_url                   NaN  ...       0.059222
qty_hyphen_url                NaN  ...       0.010346
qty_underline_url             NaN  ...       0.012158
qty_slash_url                 NaN  ...       0.001125
qty_questionmark_url          NaN  ...       0.012847
..

In [2]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
cv_scores = cross_val_score(model, x, y, cv=10)
    
print(model, ' mean accuracy: ', round(cv_scores.mean()*100, 3), '% std: ', round(cv_scores.var()*100, 3),'%')

GaussianNB()  mean accuracy:  84.286 % std:  0.001 %


In [3]:
x_train, x_test, y_train, y_test = train_test_split(x , y, test_size = 0.2)
print(x_train.shape)

(70917, 111)


In [4]:
from sklearn.model_selection import RepeatedStratifiedKFold

cv_method = RepeatedStratifiedKFold(n_splits=10, 
                                    n_repeats=3, 
                                    random_state=999)

In [6]:
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import GridSearchCV
params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}

gs_NB = GridSearchCV(estimator=model, 
                     param_grid=params_NB, 
                     cv=cv_method,
                     verbose=1, 
                     scoring='accuracy')

Data_transformed = PowerTransformer().fit_transform(x_test)

gs_NB.fit(Data_transformed, y_test);

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


Fitting 30 folds for each of 100 candidates, totalling 3000 fits


In [17]:
gs_NB.best_score_*100

90.41361158112427

In [18]:
gs_NB.best_params_

{'var_smoothing': 8.111308307896873e-08}